# Image-to-Music GenAI

Models:
* BLIP: https://huggingface.co/Salesforce/blip-image-captioning-large
* MusicGen: https://github.com/facebookresearch/audiocraft/blob/main/docs/MUSICGEN.md


In [3]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import AutoProcessor, MusicgenForConditionalGeneration
from IPython.display import Audio

In [14]:
def image_to_music(path_to_image):
  img_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
  img_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

  raw_image = Image.open(path_to_image).convert('RGB')

  # conditional image captioning
  text = "an image of"
  inputs = img_processor(raw_image, text, return_tensors="pt")

  out = img_model.generate(**inputs)
  print(img_processor.decode(out[0], skip_special_tokens=True))

  # unconditional image captioning
  inputs = img_processor(raw_image, return_tensors="pt")

  out = img_model.generate(**inputs)
  print(img_processor.decode(out[0], skip_special_tokens=True))
  txt = img_processor.decode(out[0], skip_special_tokens=True)
  audio_processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
  audio_model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

  inputs = audio_processor(
      text=[txt],
      padding=True,
      return_tensors="pt",
  )

  audio_values = audio_model.generate(**inputs, max_new_tokens=256)
  sampling_rate = audio_model.config.audio_encoder.sampling_rate
  return Audio(audio_values[0].numpy(), rate=sampling_rate)

In [15]:
image_to_music('./photo2.png')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


an image of a brain with many different types of icons
arafed image of a brain with many different types of icons


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
